In [1]:
#%%
import numpy as np 
import pandas as pd
from scipy.linalg import solve_banded, solve, solveh_banded
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt 

In [ ]:
# This code reference prof.Hwang

s0 = 1
k = 1
r = 0.03
q = 0.01
t = 0.5
vol = 0.2
optionType = 'call'
maxS = 2
N = 200
M = 2000
theta = 1

In [55]:
ds = maxS / N
dt = t / M
callOrPut = 1 if optionType.lower()=='call' else -1

i = np.arange(N+1)
j = np.arange(M+1)
s = i * ds
tt = j * dt

a = dt*(vol*s[1:-1])**2 / (2*ds**2)
b = dt*(r-q)*s[1:-1] / (2*ds)
d, m, u = a-b, -2*a-dt*r, a+b

In [56]:
A = np.diag(d[1:],-1) + np.diag(m) + np.diag(u[:-1],1)
B = np.zeros((N-1,2))
B[0,0], B[-1,1] = d[0], u[-1]

In [ ]:
Am =np.identity(N-1) - theta*A
Ap = np.identity(N-1) + (1+theta)*A
ab = np.zeros((3,N-1))
ab[0,1:] = np.diag(Am,1)
ab[1] = np.diag(Am)
ab[2,:-1] = np.diag(Am,-1)

v = np.maximum(callOrPut*(s-k), 0)
v_matrix = np.zeros((N+1,M+1))
v_matrix[:,-1] = v





In [59]:
for j in range(M-1,-1,-1):    
    #temp = Ap @ v[1:-1] + theta*B @ v[[0,-1]]
    # construct v(j+1) part which means Ap
    temp = (1-theta)*d * v[:-2] + (1 + (1-theta)*m) * v[1:-1] + (1-theta)*u * v[2:]
    # integrate v(k+1) part multiplied by (1-theta) * B
    temp += (1-theta)*B @ v[[0,-1]]
    v[0] = np.maximum(callOrPut*(0 - k * np.exp(-r * (M - j) * dt)), 0)
    v[N] = np.maximum(callOrPut*(maxS - k * np.exp(-r * (M - j) * dt)), 0)
    # integrate v(j) part multiplied by B * theta
    temp[0] += theta*d[0]*v[0]
    temp[-1] += theta*u[-1]*v[-1]
    v[1:-1] = solve_banded((1,1), ab, temp)
    v_matrix[:,j] = v


In [60]:
f = interp1d(s,v)

In [61]:
f(s0)

array(0.06086286)

In [22]:
def fdm_vanilla_option(s0, k, r, q, t, vol, optionType, maxS, N, M, theta=1):
    ds = maxS / N
    dt = t / M
    callOrPut = 1 if optionType.lower()=='call' else -1

    i = np.arange(N+1)
    s = i * ds

    a = dt*(vol*s[1:-1])**2 / (2*ds**2)
    b = dt*(r-q)*s[1:-1] / (2*ds)
    d, m, u = a-b, -2*a-dt*r, a+b

    A = np.diag(d[1:],-1) + np.diag(m) + np.diag(u[:-1],1)
    B = np.zeros((N-1,2))
    B[0,0], B[-1,1] = d[0], u[-1]

    Am = np.identity(N-1) - theta*A
    Ap = np.identity(N-1) + (1-theta)*A
    ab = np.zeros((3, N-1))
    ab[0,1:] = np.diag(Am,1)
    ab[1] = np.diag(Am)
    ab[2,:-1] = np.diag(Am,-1)

    v = np.maximum(callOrPut*(s-k), 0)
    for j in range(M-1,-1,-1):    
        #temp = Ap @ v[1:-1] + theta*B @ v[[0,-1]]
        # construct v(j+1) part which means Ap
        temp = (1-theta)*d * v[:-2] + (1 + (1-theta)*m) * v[1:-1] + (1-theta)*u * v[2:]
        # integrate v(k+1) part multiplied by (1-theta) * B
        temp += (1-theta)*B @ v[[0,-1]]
        v[0] = np.maximum(callOrPut*(0 - k * np.exp(-r * (M - j) * dt)), 0)
        v[N] = np.maximum(callOrPut*(maxS - k * np.exp(-r * (M - j) * dt)), 0)
        # integrate v(j) part multiplied by B * theta
        temp[0] += theta*d[0]*v[0]
        temp[-1] += theta*u[-1]*v[-1]
        v[1:-1] = solve_banded((1,1), ab, temp)

    f = interp1d(s,v)
    return pd.DataFrame({"S":s,"V":v}), f(s0)

In [23]:
fdm_vanilla_option(s0, k, r, q, t, vol, optionType, maxS, N, M, 0.5)

(        S             V
 0    0.00  0.000000e+00
 1    0.01  1.132392e-63
 2    0.02  5.008314e-60
 3    0.03  6.523556e-57
 4    0.04  3.972951e-54
 ..    ...           ...
 196  1.96  1.421382e+00
 197  1.97  1.446583e+00
 198  1.98  1.471819e+00
 199  1.99  1.497073e+00
 200  2.00  1.014888e+00
 
 [201 rows x 2 columns],
 array(0.060867))